<a href="https://colab.research.google.com/github/Cherevatin/ML/blob/main/Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import math
from tensorflow import keras

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.utils import np_utils, Sequence

import albumentations as A
from sklearn.metrics import accuracy_score


# Устанавливаем seed для повторяемости результатов
numpy.random.seed(42)

# Размер изображения
img_rows, img_cols = 32, 32

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Преобразование размерности изображений
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
input_shape = (img_rows, img_cols, 3)


# Нормализация данных
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.20, random_state=42)

AUGM = A.Compose([    
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.15, scale_limit=0.2, rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
])

class DataGenerator(Sequence):
 
    def __init__(self, X_data , y_data, batch_size, for_train, augm, shuffle = True):
        self.X_data = X_data
        self.y_data = y_data 
        self.batch_size = batch_size
        self.for_train = for_train
        self.augm = augm
        self.shuffle = shuffle
        self.n = 0
        self.list_IDs = numpy.arange(len(self.X_data))
        self.on_epoch_end()
        
    def __next__(self):
        # Get one batch of data
        data = self.__getitem__(self.n)
        # Batch index
        self.n += 1
        # If we have processed the entire dataset then
        if self.n >= self.__len__():
            self.on_epoch_end()
            self.n = 0        
        return data

    def __len__(self):
        # Return the number of batches of the dataset
        return math.ceil(len(self.indexes)/self.batch_size)
    
    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]        
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        
        X = self._generate_x(list_IDs_temp)
        
        if self.for_train:
            y = self._generate_y(list_IDs_temp)
            return X, y
        else:
            return X
        
    def on_epoch_end(self):
        self.indexes = numpy.arange(len(self.X_data))
        if self.shuffle: 
            numpy.random.shuffle(self.indexes)    
            
    def _generate_x(self, list_IDs_temp):
        X = numpy.zeros((self.batch_size, 32, 32, 3), dtype=numpy.float32)
        for i, ID in enumerate(list_IDs_temp):
            X[i] = self.X_data[ID]
            if self.augm:                
                img_augm = AUGM(image=X[i])
                X[i] = img_augm["image"]                
        return X
    
    def _generate_y(self, list_IDs_temp):
        #y = np.empty(self.batch_size)        
        y = numpy.zeros((self.batch_size, 10), dtype=numpy.float32)
        for i, ID in enumerate(list_IDs_temp):            
            y[i] = self.y_data[ID]            
        return y


MODELS_COUNT = 5;
y_pred = None

for i in range (MODELS_COUNT):
  # Создаем последовательную модель
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(32,32,3),padding = "same"))
  model.add(Conv2D(64, kernel_size=(4, 4),
                  activation='relu',
                   padding = "same"))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(64, (3, 3), activation='relu',padding = "same"))
  model.add(Conv2D(128, (3, 3), activation='relu'))
  model.add(Dropout(0.2)) 
  model.add(Flatten())
  model.add(Dense(500, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))


  # Компилируем модель
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  # print(model.summary())
  
  # Обучаем сеть
  # model.fit(X_train, Y_train, batch_size=200, epochs=25, validation_split=0.1, verbose=2)
  train_generator = DataGenerator(X_train, Y_train, batch_size = 200, 
                                    for_train=True, augm = True, shuffle=True)
  
  val_generator = DataGenerator(X_val, Y_val, batch_size = 200, 
                                    for_train=True, augm = False, shuffle=False)
  
  history = model.fit(train_generator,validation_data=val_generator,epochs=30, verbose=1)
  # Оцениваем качество обучения сети на тестовых данных
  scores = model.evaluate(X_test, Y_test, verbose=0)
  print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

  if y_pred is None:
    y_pred = model.predict(X_test)
  else:
    y_pred += model.predict(X_test)

y_pred /= MODELS_COUNT
pred = numpy.argmax(y_pred, axis=1)
acc_test = accuracy_score(y_test, pred)

print("Final accuracy: {}%".format(round(acc_test*100,2)))


Epoch 1/30
200/200 [==============================] - 472s 2s/step - loss: 2.0181 - accuracy: 0.2513 - val_loss: 1.4112 - val_accuracy: 0.4893
Epoch 2/30
200/200 [==============================] - 469s 2s/step - loss: 1.5078 - accuracy: 0.4537 - val_loss: 1.1775 - val_accuracy: 0.5825
Epoch 3/30
200/200 [==============================] - 468s 2s/step - loss: 1.3433 - accuracy: 0.5199 - val_loss: 1.0648 - val_accuracy: 0.6253
Epoch 4/30
200/200 [==============================] - 469s 2s/step - loss: 1.2151 - accuracy: 0.5677 - val_loss: 0.9394 - val_accuracy: 0.6646
Epoch 5/30
200/200 [==============================] - 471s 2s/step - loss: 1.1295 - accuracy: 0.5981 - val_loss: 0.9017 - val_accuracy: 0.6800
Epoch 6/30
200/200 [==============================] - 472s 2s/step - loss: 1.0702 - accuracy: 0.6198 - val_loss: 0.8268 - val_accuracy: 0.7095
Epoch 7/30
200/200 [==============================] - 471s 2s/step - loss: 1.0253 - accuracy: 0.6400 - val_loss: 0.7907 - val_accuracy: 0.7222